# Explain code and suggest code with llama7b

This notebook uses a quantized version of LLama 7B.

## Install dependencies and prepare helper code

In [ ]:
# https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers
!pip install --quiet accelerate

In [ ]:
# check GPU status
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import torch

def get_gpu_memory_usage():
  """
  Returns the current free and used memory of the GPU in megabytes.
  """
  device = torch.cuda.current_device()
  properties = torch.cuda.get_device_properties(device)
  free_memory, total_memory = torch.cuda.mem_get_info(device)
  used_memory = total_memory - free_memory
  return free_memory / 1024**2, used_memory / 1024**2

free_memory, used_memory = get_gpu_memory_usage()
print(f"Free memory: {free_memory:.2f} MB")
print(f"Used memory: {used_memory:.2f} MB")
print(torch.cuda.current_device())

Load model.

In [ ]:
from ctransformers import AutoModelForCausalLM, AutoConfig


config = AutoConfig.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF")
config.max_seq_len = 2048 #4096
config.max_answer_len= 1024

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Llama-2-7b-Chat-GGUF",
    model_file="llama-2-7b-chat.Q5_K_S.gguf",
    # model_file="llama-2-7b-chat.Q4_0.gguf",
    # model_file="llama-2-7b-chat.Q6_K.gguf",
    model_type="llama",
    gpu_layers=130,
    context_length=4096,
    max_new_tokens=2048,
  )



In [ ]:
# check that the context length has been set
llm.context_length

Render helpers

In [ ]:
from IPython.display import display, Markdown

def as_md(text):
  return display(Markdown(text))

In [ ]:
import markdown
import ipywidgets as widgets
from IPython.display import display
from typing import Callable

AnswerSetter = Callable[[str], None]

spinner = """
<div class="spinner" style="display:inline-block;position:relative;width:80px;height:40px;">
  <div
    style="display:inline-block;position:absolute;left:8px;width:16px;height:16px;border-radius:50%;background-color:#999;animation:bounce 2s infinite ease-in-out;">
  </div>
  <div
    style="display:inline-block;position:absolute;left:32px;width:16px;height:16px;border-radius:50%;background-color:#999;animation:bounce 2s infinite ease-in-out;animation-delay:-0.2s;">
  </div>
  <div
    style="display:inline-block;position:absolute;left:56px;width:16px;height:16px;border-radius:50%;background-color:#999;animation:bounce 2s infinite ease-in-out;animation-delay:-0.4s;">
  </div>
</div>
<style>
  .spinner {
    display: inline-block;
    position: relative;
    width: 80px;
    height: 80px;
  }

  .spinner div {
    display: inline-block;
    position: absolute;
    left: 8px;
    width: 16px;
    height: 16px;
    border-radius: 50%;
    background-color: #333;
    animation: bounce 2s infinite ease-in-out;
  }

  .spinner div:nth-child(2) {
    left: 32px;
    animation-delay: -0.2s;
  }

  .spinner div:nth-child(3) {
    left: 56px;
    animation-delay: -0.4s;
  }

  @keyframes bounce {

    0%,
    80%,
    100% {
      transform: scale(0);
    }

    40% {
      transform: scale(1);
    }
  }
</style>
"""

def render_q_and_a(code: str) -> AnswerSetter:
  w = widgets.HTML(value="")

  def _get_html(q: str, a: str):
    return f"""
    <table>
      <tr>
        <th style="border: 1px solid #999;">Code</th>
        <th style="border: 1px solid #999;">Explanation</th>
      </tr>
      <tr>
        <td style="padding: 2rem;">
          <code style="display: block; white-space: pre-wrap; word-wrap: break-word; background: black; padding: 1rem; text-align: left; line-height: 1.3;">{q}</code>
        </td>
        <td style="padding: 2rem; width: 60%;">
          {a}
        </td>
      </tr>
    </table>
    """

  def _answer_setter(answer: str):
    explanation_html = markdown.markdown(answer)
    w.value = _get_html(code, explanation_html)

  w.value = _get_html(code, spinner)
  display(w)

  return _answer_setter



code = """
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate

# python
def greet(name):
    print(f"Hello, {name}!")
"""

answer = """
#python

foo bar
"""

# set_answer = render_q_and_a(code)
# sleep(2)
# set_answer(answer)

## Run code explainer

In [ ]:
def describe_code(code: str) -> str:
  prompt = f"""[INST] <<SYS>>
You can explain to a humanitarian any code provided with little use of tech and math term and also without getting too much into details.
IMPORTANT: Your answers are short, precise, use markdown bullet points where appropriate and do not get too much into details. No lead-in, off topic and straight into explanation.
<</SYS>>
{code}[/INST]"""
  return llm(prompt)

In [ ]:
sample_1 = """
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(tokenize(d)) for d in df['description']])
svd = SVD(10)
svd.fit(tfidf_matrix)

def cosine(a, b):
    eps = 1e-8
    if type(a) is np.ndarray:
        return a.dot(b) / ((np.linalg.norm(a) * np.linalg.norm(b)) + eps)
    else:
        return a.dot(b) / ((a.norm() * b.norm()) + eps)

def tfidf_model(query, document):
    query_vector = tfidf_vectorizer.transform([' '.join(tokenize(query, tfidf_vectorizer.get_feature_names()))]).todense()
    doc_vector = tfidf_vectorizer.transform([' '.join(tokenize(document, tfidf_vectorizer.get_feature_names()))]).todense()
    doc_vector = np.squeeze(np.asarray(doc_vector))
    query_vector = np.squeeze(np.asarray(query_vector))
    return cosine(query_vector, doc_vector)
"""

sample_2 = """

def lsa_model(query, document):
    query = ' '.join(tokenize(query, tfidf_vectorizer.get_feature_names()))
    document = ' '.join(tokenize(document, tfidf_vectorizer.get_feature_names()))
    query_vector = tfidf_vectorizer.transform([query]).todense()
    doc_vector = tfidf_vectorizer.transform([document]).todense()
    query_vector = svd.transform(query_vector)
    doc_vector = svd.transform(doc_vector)
    return cosine(np.squeeze(np.asarray(doc_vector)), np.squeeze(np.asarray(query_vector)))

def search(model, query, documents, names = None):
    scores = [model(query, document) for document in documents]
    ixs = list(reversed(np.argsort(scores)[-10:]))

    data = [[scores[i] for i in ixs], [documents[i][:150] for i in ixs]]
    if names is not None:
        data.append([names[i] for i in ixs])
    tabulate(data, header=False)
"""
as_md(describe_code(sample_1))

In [ ]:
import requests
raw_ipynb_url = 'https://raw.githubusercontent.com/C2DH/ai-notebooks-summer-workshop/master/examples/AI_Workshop_Semantic_Search.ipynb'
ipynb_data = requests.get(raw_ipynb_url).json()

code_cells_code = [''.join(c['source']) for c in ipynb_data['cells'] if c['cell_type'] == 'code']
code_cells_code = [c for c in code_cells_code if c.replace('\n', '').strip() != '']
code_cells_code[1]

In [ ]:
for code in code_cells_code:
  set_answer = render_q_and_a(code)
  description = describe_code(code)
  set_answer(description)

## Code generator

In [ ]:
def generate_code(code: str) -> str:
  prompt = f"""[INST] <<SYS>>
You are a developer who can write Python code from an oral explanation. The code should use Impresso python library which exposes the following methods on the client instance `impresso`:
 * impresso.search.find(q=<query_term_in_any_language>, order_by=<'date', 'id', 'relevance'>)
 * impresso.articles.get(<article_id>)
All methods return a Result object with the properties:
 * `data` - result as a dict
 * `pydantic` - as a pydantic model
 * `df` - as a pandas dataframe
<</SYS>>
{code}[/INST]"""
  return llm(prompt)

In [ ]:
q = """
I need to get a list of the most recent articles that talk about nuclear power plants in German or French. I need a dataframe.
"""
code = generate_code(q)
as_md(code)